## Step 2 - Database Engineering

In [13]:
!rm hawaii.sqlite

In [14]:
# Import dependencies
import pandas as pd

#Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

#Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base

#Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date 

# To push the objects made and query the server use a Session object
from sqlalchemy.orm import Session

In [15]:
# Create a schema - Measurements and Stations classes 
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

class Station(Base):
    __tablename__ = 'station'
    id = Column (Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitute = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [16]:
# Create and establish a database connection
engine = create_engine(f'sqlite:///hawaii.sqlite')
conn = engine.connect()

# Create the measurements and stations tables within the database
Base.metadata.create_all(engine)

# # Create a Session
session = Session(bind=engine)

In [17]:
session.query(Station).all()

[]

In [18]:
#Load the cleaned csv files into a Pandas dataframe -  Where did'Unnamed column?' come from? REMOVED
measurements_df=pd.read_csv("measurements_clean_df").drop(['Unnamed: 0'], axis=1)
stations_df=pd.read_csv("stations_clean_df").drop(['Unnamed: 0'], axis=1)
measurements_df
stations_df
#print(stations_df)

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [19]:
#Create a list of data to write to_dict()
measurements_dic = measurements_df.to_dict(orient='records')
stations_dic = stations_df.to_dict(orient='records')
print (stations_dic[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [20]:
#Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [21]:
#Save references to the measurements and stations tables as variables
measurement_table = sqlalchemy.Table ('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table ('station', metadata, autoload=True)

In [22]:
#Add function to delete tables in order to re-run code multiple times without getting error
conn.execute(measurement_table.delete())
conn.execute(station_table.delete())

In [23]:
type(measurement_table.insert())

sqlalchemy.sql.dml.Insert

In [24]:
measurement_table.insert()

In [25]:
# Populate SQL table with data from csv
conn.execute(measurement_table.insert(), measurements_dic)
conn.execute(station_table.insert(), stations_dic)

In [26]:
#Test to see if tables are populated
#conn.execute('select * from station limit 5').fetchall()
conn.execute('select * from measurement limit 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]